In [1]:
#!pip install networkx

import networkx as nx
import matplotlib.pyplot as plt
import sys
import threading

In [2]:
class Graph:
    def __init__(self,adj={}):
        self.adj = adj
        self.nodes = []
        self.nodes = list(adj)
        for n in self.adj:
            self.nodes += self.adj[n]
        self.nodes = list(set(self.nodes))

        for n in self.nodes:
            if n not in self.adj:
                self.adj[n] = []
        self.rev_adj = {}
    def rename(self,new_names):
        tmp = {}
        for n in self.adj:
            tmp[new_names[n]] = self.adj[n]
        self.adj = tmp
        for n in self.adj:
            for i,u in enumerate(self.adj[n]):
                self.adj[n][i] = new_names[u]
        self.nodes = [new_names[n] for n in self.nodes]        
    def reverse(self):
        new_adj = {}
        for n in self.adj:
            if n not in new_adj:
                new_adj[n] = []
            for u in self.adj[n]:
                if u not in new_adj: 
                    new_adj[u] = []
                new_adj[u].append(n)
        self.rev_adj = new_adj        
        return Graph(new_adj)   

In [15]:
current_label = 0
numbers = {}
def DFS(g,ith_node):
    
    global current_label
    global numbers
    global explored
    global s,t
    global leader
    global times
    
    leader[ith_node] = s
    explored[ith_node] = True
    
    for v in g.adj[ith_node]:
        if not explored[v]:
            DFS(g,v)
            
    numbers[ith_node] = current_label
    current_label-=1          
    t+=1
    times[ith_node] = t

def SCCs(g):
    
    global t
    global s
    global explored
    global times
    global leader
    t = 0
    s = ''
    times = {}
    leader = {}
    explored = {}
    
    g_rev = g.reverse()
    
    for n in g_rev.nodes:
        explored[n] = False
    
    for i in sorted(g_rev.nodes,reverse=True):#range(len(g_rev.adj),0,-1):
        if not explored[i]:
            s = i
            DFS(g_rev,i)
    
    g.rename(times)
        
    for n in g.nodes:
        explored[n] = False
        
    for i in sorted(g.nodes,reverse=True):#range(len(g.adj),0,-1):
        if not explored[i]:
            s = i
            DFS(g,i)    
    
    counts = {}
    for n in leader:
        if leader[n] not in counts:
            counts[leader[n]] = 1
        else:
            counts[leader[n]] += 1
    
    counts = {k: v for k, v in sorted(counts.items(), key=lambda x: x[1])}
    counts = [v for k,v in counts.items()][::-1]
    counts = counts[:20]
    return counts    

In [16]:
f = open('SCC_test1.txt','r')
lines = f.readlines()
lines = [l.strip('\n').strip() for l in lines if l!='\n']
nodes = {}
for l in lines:#[:100000]:
    line = l.split()
    if int(line[0]) not in nodes:
        nodes[int(line[0])] = []
    for n in line[1:]:
        nodes[int(line[0])].append(int(n))

In [17]:
g = Graph(nodes)
#g.adj = nodes

In [18]:
gnx = nx.DiGraph()
for u in g.adj:
    for v in g.adj[u]:
        gnx.add_edge(u,v)
#nx.draw(gnx,with_labels=False)
#plt.show()

In [19]:
'''
gnx_rev = nx.DiGraph()
g.reverse()
for u in g.rev_adj:
    for v in g.rev_adj[u]:
        gnx_rev.add_edge(u,v)
nx.draw(gnx_rev,with_labels=False)
plt.show()
'''

'\ngnx_rev = nx.DiGraph()\ng.reverse()\nfor u in g.rev_adj:\n    for v in g.rev_adj[u]:\n        gnx_rev.add_edge(u,v)\nnx.draw(gnx_rev,with_labels=False)\nplt.show()\n'

In [20]:
import sys, threading
sys.setrecursionlimit(800000)
threading.stack_size(67108864)
def main():
    print(SCCs(g)[:20])
    
thread = threading.Thread(target=main)
thread.start()    

[3, 3, 3]


In [21]:
#SCCs(g)

In [22]:
comps = nx.algorithms.components.strongly_connected_components(gnx)

In [23]:
[len(c) for c in sorted(comps, key=len, reverse=True)][:20]

[3, 3, 3]